In [145]:
# ollama list
# ollama pull [model name]
# ollama run [model name]

# npm run 

!pip install ollama
!pip install fsspec

In [1]:
import pandas
import requests
import json
import duckdb

from ollama import chat
from ollama import ChatResponse

import importlib
import columns_to_schema

In [2]:
importlib.reload(columns_to_schema)

<module 'columns_to_schema' from '/Users/sophiawang/Desktop/CS/opendataaccessibility/backend/columns_to_schema.py'>

In [3]:
r = requests.get('https://data.cityofnewyork.us/resource/vx8i-nprf.json')
with open('database.json', 'w') as f:
    json.dump(r.json(), f)

schema = columns_to_schema.form_schema("data.cityofnewyork.us", "vx8i-nprf", "db")
print(schema)

CREATE TABLE db (
	exam_no VARCHAR, --A four (4) digit number that identifies a civil service examination. 
	list_no FLOAT, --An eligible candidate’s placement on a given list as of the business day that the data set is populated. 
	first_name VARCHAR, --A candidate’s first name as it appears on their application.
	mi VARCHAR, --A candidate’s middle initial (MI) as it appears on their application.
	last_name VARCHAR, --A candidate’s last name as it appears on their application.
	adj_fa FLOAT, --The Adjusted Final Average (“Adj. FA” or “AFA”) is an eligible candidate’s test score in addition to any additional credits granted.
	list_title_code VARCHAR, --A five (5) digit number that corresponds to and represents a civil service title.
	list_title_desc VARCHAR, --A descriptive name that identifies a civil service title. 
	group_no VARCHAR, --Represents the certification order for a list. (e.g. – eligible candidates on a list with a 01 group number may be considered for appointment before 

In [157]:
question = 'give me the first names of all candidates with the middle initial A'

prompt = f"""
Generate a SQL query to answer this question: [QUESTION]{question}[/QUESTION]. Do not change the prompt. DO NOT ADD

Write in plain text without any formatting.

The query will run on a database with the following schema:
{schema}

Answer: Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]. [SQL] <s>
"""

response: ChatResponse = chat(model='sqlcoder:latest', messages=[
  {
    'role': 'user',
    'content': prompt,
  },
])

output = response['message']['content']
print(output)

 SELECT first_name FROM db WHERE mi = 'A';


In [158]:
db = duckdb.read_json('database.json')
duckdb.sql(output)

# https://duckdb.org/docs/stable/clients/python/overview.html 

┌────────────┐
│ first_name │
│  varchar   │
├────────────┤
│ JOSEPH     │
│ TESHAWNA   │
│ TYRESE     │
│ ALIYAH     │
│ DEVENN     │
│ SERGIO     │
│ JASON      │
│ MD         │
│ TASHEEMA   │
│ CARLOS     │
│ ·          │
│ ·          │
│ ·          │
│ MD         │
│ JASMINE    │
│ ANDREW     │
│ JOSHUA     │
│ JAMES      │
│ SHANYIA    │
│ KRISTOPHER │
│ KENEIL     │
│ DANIELLA   │
│ FRANCO     │
├────────────┤
│  87 rows   │
│ (20 shown) │
└────────────┘